<a href="https://colab.research.google.com/github/GrandWizard1102/NM_projects/blob/main/Pdf_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import gradio as gr
import faiss
import numpy as np
import fitz  # PyMuPDF for PDF text extraction
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory  # Simpler memory
from langchain.text_splitter import RecursiveCharacterTextSplitter
import getpass


# ✅ Load Sentence Transformer for Embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embedding_dim = 384

# ✅ FAISS Vector Store
index = faiss.IndexFlatL2(embedding_dim)
stored_texts = []

# ✅ LLM Model (OpenRouter)
API_KEY = getpass.getpass("Enter your OpenRouter API key: ")
BASE_URL = "https://openrouter.ai/api/v1"

llm = ChatOpenAI(
    model="mistralai/mistral-small",
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

# ✅ Memory for Context
memory = ConversationBufferMemory(memory_key="chat_history", input_key="query")

# Clear memory after every 5 queries to refresh context
if len(memory.load_memory_variables({})['chat_history']) > 5:
    memory.clear()


# ✅ Convert Text to Embeddings
def embed_text(text):
    return embedder.encode([text])[0]

# ✅ Add Text to FAISS
def add_to_index(text):
    vector = embed_text(text)
    index.add(np.array([vector], dtype=np.float32))
    stored_texts.append(text)

# ✅ Retrieve Similar Text from PDFs
def retrieve_similar_text(query, top_k=3):
    if len(stored_texts) == 0:
        return ["No relevant data found."]

    query_vector = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    return [stored_texts[i] for i in indices[0] if i < len(stored_texts)]

# ✅ Create Prompt Template
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
    You are an AI assistant answering questions based on uploaded PDFs.
    Context: {context}
    Question: {query}
    Answer:
    """
)

# ✅ Create LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# ✅ Function to Extract Text from PDFs
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# ✅ Function to Upload PDF and Store Embeddings
def process_pdf(file):
    pdf_text = extract_text_from_pdf(file)

    # ✅ Improved Chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=50
    )
    chunks = text_splitter.split_text(pdf_text)

    for chunk in chunks:
        add_to_index(chunk)

    return "PDF processed successfully! Now you can ask questions."

# ✅ Generate Response
def generate_response(query):
    contexts = retrieve_similar_text(query)
    context_str = "\n\n---\n\n".join(contexts) if contexts else "No relevant data found."

    response = llm_chain.invoke({"context": context_str, "query": query})

    return f"<pre>{response}</pre>"  # Preserves formatting




# ✅ Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 📄 PDF-Based Chatbot")

    file_upload = gr.File(label="Upload PDF", type="filepath")

    upload_btn = gr.Button("Process PDF")
    output_text = gr.Textbox(label="PDF Processing Status", interactive=False)

    query_input = gr.Textbox(label="Ask a question")
    query_btn = gr.Button("Submit")
    answer_output = gr.Textbox(label="Answer", interactive=False)

    upload_btn.click(process_pdf, inputs=file_upload, outputs=output_text)
    query_btn.click(generate_response, inputs=query_input, outputs=answer_output)

demo.launch(debug=True)


Enter your OpenRouter API key: ··········
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://09bf7326d5deb7059d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://09bf7326d5deb7059d.gradio.live


In [2]:
pip install gradio faiss-cpu numpy pymupdf sentence-transformers langchain_community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


**Medical Report**

In [10]:
import gradio as gr
import faiss
import numpy as np
import fitz  # PyMuPDF for PDF text extraction
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# ✅ Load Sentence Transformer for Medical Embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embedding_dim = 384

# ✅ FAISS Vector Store
index = faiss.IndexFlatL2(embedding_dim)
stored_texts = []

# ✅ LLM Model (OpenRouter) - Add Your API Key
API_KEY = getpass.getpass("Enter your OpenRouter API key: ")  # Replace with your actual key
BASE_URL = "https://openrouter.ai/api/v1"


from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="mistralai/mistral-small",
    openai_api_base=BASE_URL,
    openai_api_key=API_KEY,
    request_timeout=60,
    default_headers={"Authorization": f"Bearer {API_KEY}"}  # ✅ OpenRouter authentication
)


# ✅ Memory for Context
memory = ConversationBufferMemory(memory_key="chat_history", input_key="query")

# ✅ Convert Text to Embeddings
def embed_text(text):
    return embedder.encode([text])[0]

# ✅ Add Text to FAISS
def add_to_index(text):
    vector = embed_text(text)
    index.add(np.array([vector], dtype=np.float32))
    stored_texts.append(text)

# ✅ Retrieve Similar Text from Medical Reports
def retrieve_similar_text(query, top_k=3):
    if len(stored_texts) == 0:
        return ["No relevant data found."]

    query_vector = embed_text(query).reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    return [stored_texts[i] for i in indices[0] if i < len(stored_texts)]

# ✅ Create Medical Chatbot Prompt
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
    You are a medical AI assistant analyzing a patient's medical report.
    Based on the provided medical data, answer the user's query accurately.

    Medical Report Context:
    {context}

    Patient Query:
    {query}

    Medical AI Response:
    """
)

# ✅ Create LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# ✅ Function to Extract Text from Medical Reports
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# ✅ Function to Upload and Process Medical Reports
def process_pdf(file):
    pdf_text = extract_text_from_pdf(file)

    # ✅ Chunking for Better Processing
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=50
    )
    chunks = text_splitter.split_text(pdf_text)

    for chunk in chunks:
        add_to_index(chunk)

    return "Medical report processed successfully! Now you can ask questions."

# ✅ Generate Response
def generate_response(query):
    contexts = retrieve_similar_text(query)  # Retrieve relevant medical info
    context_str = "\n\n---\n\n".join(contexts) if contexts else "No relevant data found."

    response = llm_chain.invoke({"context": context_str, "query": query})
    return response

# ✅ Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 🏥 AI Healthcare Chatbot")

    file_upload = gr.File(label="Upload Medical Report (PDF)", type="filepath")
    upload_btn = gr.Button("Process Report")
    output_text = gr.Textbox(label="Processing Status", interactive=False)

    query_input = gr.Textbox(label="Ask a Medical Question")
    query_btn = gr.Button("Submit")
    answer_output = gr.Textbox(label="AI Medical Response", interactive=False)

    upload_btn.click(process_pdf, inputs=file_upload, outputs=output_text)
    query_btn.click(generate_response, inputs=query_input, outputs=answer_output)

demo.launch(debug=True)


Enter your OpenRouter API key: ··········
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6c8fe71a82ec419d8b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6c8fe71a82ec419d8b.gradio.live
